<a href="https://colab.research.google.com/github/offdocudev/Machine-Learning/blob/master/fr_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import math
from IPython.display import clear_output

# DATA - https://www.kaggle.com/c/digit-recognizer/data
# Kaggle 
! pip install kaggle
! mkdir ~/.kaggle
#! cp kaggle.json ~/.kaggle/
# Or manually copy credentials in kaggle.json file 

!  echo '{"username":"offdocudev","key":"a6893bc3aa38aec8a688959083e82644"}' > /root/.kaggle/kaggle.json


! cat /root/.kaggle/kaggle.json
! chmod 600 /root/.kaggle/kaggle.json

#################### DOWNLOAD AND UNZIP FILE SAVED IN DRIVE ####################

!pip install -U -q PyDrive

# HERE YOUR FILE ID ( GET IT WITH THE SHARING URL: https://drive.google.com/open?id=1Soh3zXLXt2lT7b_3FcWWyeOCC7SnOxK0 )
zip_id = '1Soh3zXLXt2lT7b_3FcWWyeOCC7SnOxK0'

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import zipfile, os

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)




Using TensorFlow backend.


mkdir: cannot create directory ‘/root/.kaggle’: File exists
{"username":"offdocudev","key":"a6893bc3aa38aec8a688959083e82644"}


In [0]:
!kaggle datasets download -d jannesklaas/frenchenglish-bilingual-pairs

frenchenglish-bilingual-pairs.zip: Skipping, found more recently modified local copy (use --force to force download)


In [0]:
zip_ref = zipfile.ZipFile('frenchenglish-bilingual-pairs.zip','r')
zip_ref.extractall()
zip_ref.close()

In [0]:
zip_ref = zipfile.ZipFile('fra-eng.zip','r')
zip_ref.extractall()
zip_ref.close()

In [0]:
import os

In [0]:
print(os.listdir('/content'))

['.config', '_about.txt', 'fra.txt', 'fra-eng.zip', 'frenchenglish-bilingual-pairs.zip', 'adc.json', 'sample_data']


In [0]:

from keras.layers import CuDNNLSTM,Dense,Dropout

In [0]:
data_path = 'fra.txt'

In [0]:
df = pd.read_csv(data_path,delimiter= '\t')

In [0]:
lines = open(data_path).read().split('\n')

In [0]:
num_samples = 10000
input_texts =[]
target_texts = []
input_characters = set()
target_characters = set()



In [0]:
for line in lines[:min(num_samples,len(lines) -1)]:
  
  input_text,target_text = line.split('\t')
  input_texts.append(input_text)
  target_text = '\t' + target_text + '\n'
  target_texts.append(target_text)
  
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
      
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)
    

In [0]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

num_encoder = len(input_characters)
num_decoder = len(target_characters)

print(num_encoder,num_decoder)

input_characters


71 93


[' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '’']

In [0]:
input_index = {char : i for i,char in enumerate(input_characters)}
target_index = {char : i for i,char in enumerate(target_characters)}

In [0]:
max_encoder = max([len(txt)  for txt in input_texts])
max_decoder = max([len(txt) for txt in target_texts])


In [0]:
print(max_encoder,max_decoder)

16 59


In [0]:
encoder_input_data = np.zeros((len(input_texts),max_encoder,num_encoder),dtype='float32')
decoder_input_data = np.zeros((len(input_texts),max_decoder,num_decoder),dtype='float32')
decoder_target_data = np.zeros((len(target_texts),max_decoder,num_decoder),dtype='float32')


In [0]:
for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
  for t,char in enumerate(input_text):
    encoder_input_data[i,t,input_index[char]] = 1
    
  for t,char in enumerate(target_text):
    
    decoder_input_data[i,t,target_index[char]] = 1
    
    if t>0:
      decoder_target_data[i,t-1,target_index[char]] =1

In [0]:
from keras.models import Model
from keras.layers import Dense,LSTM,CuDNNLSTM,Input
latent_dim =512

In [0]:
encoder_input = Input(shape = (None,num_encoder))

encoder = CuDNNLSTM(latent_dim,return_state=True)
encoder_output,state_h,state_c = encoder(encoder_input)
encoder_states = [state_h,state_c]


Instructions for updating:
Colocations handled automatically by placer.


In [0]:
decoder_input = Input(shape = (None,num_decoder))
decoder = CuDNNLSTM(latent_dim,return_state=True,return_sequences=True)
decoder_output,_,_ = decoder(decoder_input,initial_state = encoder_states)

decoder_dense = Dense(num_decoder,activation='softmax',)
decoder_output = decoder_dense(decoder_output)

In [0]:
model = Model([encoder_input,decoder_input],decoder_output)

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics =['acc'])
history = model.fit([encoder_input_data, decoder_input_data], 
                    decoder_target_data,
                    batch_size=1024,
                    epochs=100,
                    validation_split=0.2)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 8000 samples, validate on 2000 samples
Epoch 1/100
8000/8000 [==============================] - 9s 1ms/step - loss: 1.2267 - acc: 0.0328 - val_loss: 1.1875 - val_acc: 0.0567
Epoch 2/100
8000/8000 [==============================] - 4s 447us/step - loss: 0.9843 - acc: 0.0498 - val_loss: 1.1106 - val_acc: 0.0826
Epoch 3/100
8000/8000 [==============================] - 4s 438us/step - loss: 0.9517 - acc: 0.0565 - val_loss: 1.0954 - val_acc: 0.0711
Epoch 4/100
8000/8000 [==============================] - 4s 472us/step - loss: 0.9385 - acc: 0.0566 - val_loss: 1.1013 - val_acc: 0.0700
Epoch 5/100
8000/8000 [==============================] - 4s 463us/step - loss: 0.9271 - acc: 0.0606 - val_loss: 1.1008 - val_acc: 0.0572
Epoch 6/100
8000/8000 [==============================] - 4s 484us/step - loss: 0.9159 - acc: 0.0631 - val_loss: 1.0797 - val_acc: 0.0759
Epoch 

In [0]:
encoder_model = Model(encoder_input,encoder_states)

In [0]:
decoder_input_h = Input(shape = (latent_dim,))
decoder_input_c = Input(shape = (latent_dim,))

decoder_s_inputs = [decoder_input_h,decoder_input_c]

decoder_output,states_h,states_c = decoder(decoder_input ,initial_state=decoder_s_inputs)
decoder_states = [states_h,states_c]

decoder_output = decoder_dense(decoder_output)

decoder_model = Model([decoder_input]+decoder_s_inputs,[decoder_output] + decoder_states)

In [0]:
reverse_input_index = {i:char for char,i in input_index.items() } 
reverse_target_index = {i:char for char,i in target_index.items() } 

In [0]:
def decoder_sequence(input_seq):
  states_value = encoder_model.predict(input_seq)
  
  target_seq = np.zeros((1,1,num_decoder))
  
  target_seq[0,0,target_index['\t']] = 1
  
  stop_condition = False
  decoded_sentence = ''
  
  while not stio_condition :
    output_tokens,h,c = decoder_model.predict([target_seq] +states_value)
    
    sampled_token_index = np.argmax(output_tokens[0,-1,:])
    
    sampled_char = reverse_target_index[sampled_token_index]
    
    decoded_sentence += sampled_char
    
    if(sampled_char == '\n' or len(decoded_sentence) > max_decoder ):
      stop_condition = True
      
    target_seq = np.zeros((1,1,num_decoder))
    target_sq[0,0,sampled_token_index] = 1.
    
    states_value = [h,c]
    
    
    return decoded_sentence

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    
    # Loop untill we recieve a stop sign
    while not stop_condition:
        # Get output and internal states of the decoder 
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Get the predicted token (the token with the highest score)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        # Get the character belonging to the token
        sampled_char = reverse_target_char_index[sampled_token_index]
        # Append char to output
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


In [0]:
my_text = 'Thanks'
placeholder = np.zeros((1,len(my_text)+10,num_encoder_tokens))

In [0]:
for i, char in enumerate(my_text):
    print(i,char, input_token_index[char])
    placeholder[0,i,input_token_index[char]] = 1

0 T 31
1 h 42
2 a 35
3 n 48
4 k 45
5 s 53


In [0]:
decode_sequence(placeholder)

ValueError: ignored